In [ ]:
#@title  { run: "auto", display-mode: "form" }
student_id = "80764201" #@param {type:"string"}
first_name = "Jorge" #@param {type:"string"}
last_name = "Gonzalez" #@param {type:"string"}

In [ ]:
class Author:
    def __init__(self, name, birth_year):
        self.name = name
        self.birth_year = birth_year
        self.books = set()

    def add_book(self, book):
      #adds book into the book set that is inside the author class
        self.books.add(book)


In [ ]:
class Book:
    def __init__(self, isbn, title, author, year, copies, genre):
        self.isbn = isbn
        self.title = title
        self.author = author
        self.year = year
        self.copies = copies
        self.available_copies = copies
        self.genre = genre

    def __str__(self):
      # obtain the attributes from the class and print them in a organized way for the user
        return f"{self.title} by {self.author.name} ({self.year})"

In [ ]:
# import the library to obtain current date
from datetime import date

class Customer:
    def __init__(self, customer_id, name, email):
        self.customer_id = customer_id
        self.name = name
        self.email = email
        self.borrowed_books = []

    def borrow_book(self, book):
      # if the book is not in the borrowed_books then we add it into it with the current date to keep track for the late returns
        if book not in self.borrowed_books:
            self.borrowed_books[book] = date.today()
        else:
            print(f"Book is already borrowed by {self.name}")


    def return_book(self, book):
        # once the method is called upon the desired book will be removed
        self.borrowed_books.remove(book)

    def get_borrowed_books(self):
        print(self.borrowed_books)

In [ ]:
# import the library to read from the Books.csv file
import csv

class LibraryManagementSystem:
    # method to help read the given Books.csv file and populate our library database
    def load_books_from_csv(self, filename):
        try:
            with open(filename, newline='', encoding='utf-8') as csvfile:
                reader = csv.DictReader(csvfile)

                for row in reader:
                    isbn = row['ISBN']
                    title = row['Title']
                    author_name = row['Author Name']
                    author_birth_year = int(row['Author Birth Year'])
                    year = int(row['Year'])
                    copies = int(row['Copies'])
                    genre = row['Genre']

                    # Add the book to the library
                    self.add_book(isbn, title, author_name, author_birth_year, year, copies, genre)
            print("Books loaded successfully from the CSV file.")

        except Exception as e:
            print(f"Error loading books from CSV: {e}")

    def __init__(self):
        self.books = {}  # Dictionary: ISBN -> Book object
        self.authors = {}  # Dictionary: name -> Author object
        self.customers = {}  # Dictionary: customerID -> Customer object
        self.genre_classification = {}  # Dictionary: Genre -> {set of ISBNs}
        self.waitlist = {}  # Dictionary: ISBN -> [list of customerIDs]

    def add_book(self, isbn, title, author_name, author_birth_year, year, copies, genre):
        # checking if the author isn't already listed
        if author_name not in self.authors:
          # if not, then we add the new author to the list
          new_author = Author(author_name, author_birth_year)
          self.authors[author_name] = new_author
        else:
          # if yes, then we obtain the existing author
          new_author = self.authors[author_name]

        # creating the new book object
        new_book = Book(isbn, title, new_author, year, copies, genre)
        # adding it into the author's collection
        new_author.add_book(new_book)
        # adding the book into the dictionary using the ISBN as the key
        self.books[isbn] = new_book

        # updating the genre_classification dictionary
        if genre not in self.genre_classification:
            self.genre_classification[genre] = set()
        self.genre_classification[genre].add(isbn)

    def register_customer(self, name, email):

        # creating the new customer and providing them with a unique id
        new_customer_id = len(self.customers) + 1
        new_customer = Customer(new_customer_id, name, email)
        self.customers[new_customer_id] = new_customer
        return new_customer_id

    def borrow_book(self, isbn, customer_id):
      # checks if the book and customer exist in the system
      if isbn not in self.books or customer_id not in self.customers:
        return f"Book {isbn} or Customer ID {customer_id} not found"

      # retrives the book and customer objects using the ID
      book = self.books[isbn]
      customer = self.customers[customer_id]

      # checks if the book has any available copies for borrowing
      if book.available_copies > 0:
          book.available_copies -= 1
          customer.borrow_book(book)
          return f"Book {book.title} borrowed by Customer {customer_id}"
      else:
          return self.add_to_waitlist(isbn, customer_id)

    def return_book(self, isbn, customer_id):
        #checks if the book and the customer are in the system
        if isbn not in self.books or customer_id not in self.customers:
          return f"Book {isbn} or Customer ID {customer_id} not found"

        # retrieves the book and customer objects
        book = self.books[isbn]
        customer = self.customers[customer_id]

        # checks if the customer has borrowed this book
        if book in customer.borrowed_books:
          book.available_copies += 1
          customer.return_book(book)

          # if there are customers on the waitlist, borrow the book to the next customer
          if isbn in self.waitlist and self.waitlist[isbn]:
              next_customer_id = self.waitlist[isbn].pop(0)
              return self.borrow_book(isbn, next_customer_id)
        else:
          return f"Customer {customer_id} has not borrowed book {isbn}"

    def search_books(self, query):
        # Initialize a list to store search results
        results = []
        # convert the search query to lowercase for case-insensitive matching
        query = query.lower()

        # Search by title, author, or ISBN
        for book in self.books.values():
            if query in book.title.lower() or query in book.author.name.lower() or query in book.isbn:
                results.append(book)

        return results

    def display_available_books(self):
        # filters through the books with available copies
        available_books = [book for book in self.books.values() if book.available_copies > 0]
        return available_books

    def display_customer_books(self, customer_id):
        # check if the customer exists in the system
        if customer_id in self.customers:
            customer = self.customers[customer_id]
            # returns the list of the borrowed book by the customer
            return customer.get_borrowed_books()
        else:
            return f"Customer ID {customer_id} not found"

    def recommend_books(self, customer_id):
      # checks for customer in the system
        if customer_id not in self.customers:
          return f"Customer ID {customer_id} not found"

        customer = self.customers[customer_id]
        # gets unique genres to find the list of recommended books
        borrowed_genres = {book.genre for book in customer.borrowed_books}

        recommended_books = []
        # loop through the gernes to find recommended books
        for genre in borrowed_genres:
          for isbn in self.genre_classification.get(genre, []):
            book = self.books[isbn]

            # checks if the book is available and not already in the customer's borrowed books
            if book.available_copies > 0 and book not in customer.borrowed_books:
                recommended_books.append(book)
                # limits recommendations to 5
            if len(recommended_books) >= 5:
                break

        return recommended_books

    def add_to_waitlist(self, isbn, customer_id):
        # checking if the isbn isn't already added
        if isbn not in self.waitlist:
            self.waitlist[isbn] = []
        # add the customer to the waiting list if not already in it
        if customer_id not in self.waitlist[isbn]:
            self.waitlist[isbn].append(customer_id)
            return f"Customer {customer_id} added to waitlist for book {isbn}"
        else:
            return f"Customer {customer_id} is already in the waitlist for book {isbn}"

    def check_late_returns(self, days_threshold=14):
        # initialize a list to store late return entries
        from datetime import timedelta
        lateReturns = []

        # Iterate over all customers to check for late returns
        for customer in self.customers.values():
          for book, borrow_date in customer.borrowed_books.item():
            # cheks if the number of days since the book was borrowed passes the threshold
            if date.today() - borrow_date > timedelta(days=days_threshold):
              lateReturns.append(book)

        return lateReturns

    def run(self):
      # loads books from the csv file
      self.load_books_from_csv('/content/Books.csv')

      while True:
          print("1. Add Book")
          print("2. Register Customer")
          print("3. Borrow Book")
          print("4. Return Book")
          print("5. Search Books")
          print("6. Display Available Books")
          print("7. Display Customer's Borrowed Books")
          print("8. Recommend Books")
          print("9. Check Late Returns")
          print("10. Exit")

          choice = input("Select an option (1-10): ")

          if choice == "1":
              # prompts for the user to add a book into the libary database
              isbn = input("Enter ISBN: ")
              title = input("Enter title: ")
              author_name = input("Enter author name: ")
              author_birth_year = int(input("Enter author birth year: "))
              year = int(input("Enter year: "))
              copies = int(input("Enter number of copies: "))
              genre = input("Enter genre: ")
              self.add_book(isbn, title, author_name, author_birth_year, year, copies, genre)
              print("Book added successfully!")

          elif choice == "2":
              # Logic to register a customer
              name = input("Enter customer name: ")
              email = input("Enter customer email: ")
              customer_id = self.register_customer(name, email)
              print(f"Customer registered with ID: {customer_id}")

          elif choice == "3":
              # Logic to borrow a book
              isbn = input("Enter ISBN of the book to borrow: ")
              customer_id = int(input("Enter customer ID: "))
              result = self.borrow_book(isbn, customer_id)
              print(result)

          elif choice == "4":
              # Logic to return a book
              isbn = input("Enter ISBN of the book to return: ")
              customer_id = int(input("Enter customer ID: "))
              result = self.return_book(isbn, customer_id)
              print(result)

          elif choice == "5":
              # Logic to search for books
              query = input("Enter search query: ")
              results = self.search_books(query)
              if results:
                  print("Search results:")
                  for book in results:
                      print(book)
              else:
                  print("No matching books found.")

          elif choice == "6":
              # Display available books
              available_books = self.display_available_books()
              if available_books:
                  print("Available Books:")
                  for book in available_books:
                      print(book)
              else:
                  print("No available books at the moment.")

          elif choice == "7":
              # Display a customer's borrowed books
              customer_id = int(input("Enter customer ID: "))
              borrowed_books = self.display_customer_books(customer_id)

          elif choice == "8":
              # Recommend books
              customer_id = int(input("Enter customer ID: "))
              recommended_books = self.recommend_books(customer_id)
              if recommended_books:
                  print("Recommended Books:")
                  for book in recommended_books:
                      print(book)
              else:
                  print("No recommended books at the moment.")

          elif choice == "9":
              # Check for late returns
              late_returns = self.check_late_returns()
              if late_returns:
                  print("Late Returns:")
                  for book, customer_id in late_returns:
                      print(f"Book: {book}, Customer ID: {customer_id}")
              else:
                  print("No late returns at the moment.")

          elif choice == "10":
              print("Exiting the system.")
              break
          else:
              print("Invalid choice. Please select a number between 1 and 10.")

if __name__ == "__main__":
    lms = LibraryManagementSystem()
    lms.run()

Books loaded successfully from the CSV file.
1. Add Book
2. Register Customer
3. Borrow Book
4. Return Book
5. Search Books
6. Display Available Books
7. Display Customer's Borrowed Books
8. Recommend Books
9. Check Late Returns
10. Exit
Select an option (1-10): 10
Exiting the system.


## Deadlines

1. Check-in - September 24, 2024 at 11:59pm
    *   Your code will not be graded, however, this check-in is required to earn credit on the lab. You will submit your progress up to this point. You will be evaluated only on the amount of progress that you have made (i.e., 1 - satisfactory progress, 0-unsatisfactory progress); solutions do not yet need to be complete, but should be reasonably progressed.
    *   Submit your progress by uploading the .ipynb file on Blackboard under "Lab 1 - Check-In"
2. Final Submission - October 1, 2024 at 11:59pm
    *   You should submit a completed lab assignment to include all appropriate source code above.



## How to Submit

1. File > Download .ipynb
2. Go to Blackboard, find the submission page, and upload the .ipynb file you just downloaded.
3. Submit the PDF of your lab report on Blackboard